# Operaciones CRUD en Riak con Python

## Conexión a Riak

In [1]:
import riak

In [2]:
cliente = riak.RiakClient()
print("¿Está Riak disponible?", cliente.ping())

¿Está Riak disponible? True


In [3]:
!curl -s http://localhost:8098/stats  | jq .riak_kv_version

"2.1.7-0-gbd8e312"


## Insertar valores

In [4]:
bucket = cliente.bucket("bucket_name")

In [5]:
key1 = "key_one"
val1 = 1
key = bucket.new(key1, data = val1)
key.store()

In [6]:
key2 = "key_two"
val2 = "Dos"
key = bucket.new(key2, data = val2)
key.store()

In [7]:
key3 = "key_three"
val3 = {
    "val1" : 1,
    "val2" : "Dos",
    "val3" : [1, 2, 3, 4, 5],
    
}

In [8]:
key = bucket.new(key3, data = val3)
key.store()

## Insertando imágenes

In [9]:
image_bucket = cliente.bucket('images')

photo = image_bucket.new_from_file('image_key', './images/demo.image.jpg')
photo.store()

## Recuperando información

In [10]:
from pprintpp import pprint as pp
bucket = cliente.bucket("bucket_name")

print("key_one:", bucket.get("key_one").data)
print("key_two:", bucket.get("key_two").data)
data3 = bucket.get("key_three").data

print("key_three: ")
pp(data3)

key_one: 1
key_two: Dos
key_three: 
{'val1': 1, 'val2': 'Dos', 'val3': [1, 2, 3, 4, 5]}


In [11]:
!curl http://localhost:8098/riak/bucket_name/key_one

1

In [12]:
!curl http://localhost:8098/riak/bucket_name/key_two

"Dos"

In [13]:
!curl -s http://localhost:8098/riak/bucket_name/key_three | jq

{
  "val1": 1,
  "val2": "Dos",
  "val3": [
    1,
    2,
    3,
    4,
    5
  ]
}


http://localhost:8098/riak/images/image_key

# Tablas

## Escritura en Riak

Vamos a crear un dataset de datos ficticio y lo vamos a cargar un dataframe de Pandas

In [14]:
from faker import Faker
import pandas as pd
import json

faker = Faker()
Faker.seed(4321)
profiles = []
for i in range(10):
    profile = faker.simple_profile()
    profile['birthdate'] = profile['birthdate'].isoformat()
    profiles.append(profile) 

dataframe = pd.read_json(json.dumps(profiles, default=str))

In [15]:
dataframe

,username,name,sex,address,mail,birthdate
0,daniel02,Robert Duffy,M,"6936 Griffin Extension Suite 018\nSouth Jim, S...",annahamilton@gmail.com,2020-10-01
1,emyers,David Moody,M,"335 Gloria Street Suite 675\nMathisport, NJ 73764",webstermadeline@hotmail.com,1947-05-12
2,fosterroberto,Nicole Jones,F,"84119 Fisher Vista\nEast Cherylton, MS 37193",amandacunningham@yahoo.com,2019-01-19
3,masonnguyen,Scott Lucas,M,"6401 Jeff Ford\nAngelicashire, WV 53209",adam32@hotmail.com,1989-03-03
4,jamie41,Brittany Miller,F,"2871 Mary Ways Apt. 812\nWest Coryhaven, MI 11710",nancy38@yahoo.com,2008-04-07
5,kmartin,Dwayne Miller,M,"56403 Misty Knolls Suite 206\nMartinstad, ID 6...",gregory32@hotmail.com,2016-10-11
6,sarahdawson,Brandon Adams,M,"873 Amber Underpass\nMartinezfurt, AK 85986",joshua01@hotmail.com,1972-05-12
7,andersoncheryl,Ryan Roberts,M,"1210 Juarez Crest Suite 970\nSolisburgh, DC 38745",darrylbanks@yahoo.com,1949-06-15
8,ghardy,Mr. Gary Griffin,M,"65885 Amber Locks Apt. 633\nNorth Steven, ND 0...",brenda14@yahoo.com,1981-12-25
9,jcherry,Andrew Martinez,M,14704 Jennifer Extensions Suite 115\nNorth Ant...,andrewhowell@hotmail.com,1948-03-08


Cada una de las filas del dataset es JSON (diccionario en Python)

In [16]:
profiles[0]

{'username': 'daniel02',
 'name': 'Robert Duffy',
 'sex': 'M',
 'address': '6936 Griffin Extension Suite 018\nSouth Jim, SD 34941',
 'mail': 'annahamilton@gmail.com',
 'birthdate': '2020-10-01'}

In [17]:
print("username:", profiles[0]['username'])
print("ser:", profiles[0]['sex'])
print("mail:", profiles[0]['mail'])

username: daniel02
ser: M
mail: annahamilton@gmail.com


Cargamos la información en Riak en un backet llamado 'profile'-

Adicionalmente creamos 2 indices, uno por sexo y otro por email.

Los índices en Riak permiten buscar clavés de una manerá mucho más rápida que si recorrieramos todas las claves de un bucket una tras otra de forma secuencial

In [18]:
bucket = cliente.bucket("profile")

for profile in profiles:
    riak_key = bucket.new(profile['username'], data = profile)
    riak_key.add_index('idx_mail_bin', profile['mail'])
    riak_key.add_index('idx_sex_bin', profile['sex'])
    riak_key.store()

## Lectura


Obtenemos todas los valores de la tabla y cargamos la información en Pandas

In [19]:
rows = []
for keys in bucket.stream_keys():
    for key in keys:
        rows.append(bucket.get(key).data)

df = pd.read_json(json.dumps(rows))
df

,username,name,sex,address,mail,birthdate
0,masonnguyen,Scott Lucas,M,"6401 Jeff Ford\nAngelicashire, WV 53209",adam32@hotmail.com,1989-03-03
1,emyers,David Moody,M,"335 Gloria Street Suite 675\nMathisport, NJ 73764",webstermadeline@hotmail.com,1947-05-12
2,daniel02,Robert Duffy,M,"6936 Griffin Extension Suite 018\nSouth Jim, S...",annahamilton@gmail.com,2020-10-01
3,kmartin,Dwayne Miller,M,"56403 Misty Knolls Suite 206\nMartinstad, ID 6...",gregory32@hotmail.com,2016-10-11
4,fosterroberto,Nicole Jones,F,"84119 Fisher Vista\nEast Cherylton, MS 37193",amandacunningham@yahoo.com,2019-01-19
5,andersoncheryl,Ryan Roberts,M,"1210 Juarez Crest Suite 970\nSolisburgh, DC 38745",darrylbanks@yahoo.com,1949-06-15
6,jcherry,Andrew Martinez,M,14704 Jennifer Extensions Suite 115\nNorth Ant...,andrewhowell@hotmail.com,1948-03-08
7,sarahdawson,Brandon Adams,M,"873 Amber Underpass\nMartinezfurt, AK 85986",joshua01@hotmail.com,1972-05-12
8,jamie41,Brittany Miller,F,"2871 Mary Ways Apt. 812\nWest Coryhaven, MI 11710",nancy38@yahoo.com,2008-04-07
9,ghardy,Mr. Gary Griffin,M,"65885 Amber Locks Apt. 633\nNorth Steven, ND 0...",brenda14@yahoo.com,1981-12-25


Ahora vamos a localizar las claves del sexo "M" a través del ínidice

In [20]:
rows = []
keys = bucket.get_index("idx_sex_bin", 'M')
for key in keys.results:
    row = bucket.get(key).data
    rows.append(row)
       
df = pd.read_json(json.dumps(rows))
df

,username,name,sex,address,mail,birthdate
0,daniel02,Robert Duffy,M,"6936 Griffin Extension Suite 018\nSouth Jim, S...",annahamilton@gmail.com,2020-10-01
1,masonnguyen,Scott Lucas,M,"6401 Jeff Ford\nAngelicashire, WV 53209",adam32@hotmail.com,1989-03-03
2,kmartin,Dwayne Miller,M,"56403 Misty Knolls Suite 206\nMartinstad, ID 6...",gregory32@hotmail.com,2016-10-11
3,andersoncheryl,Ryan Roberts,M,"1210 Juarez Crest Suite 970\nSolisburgh, DC 38745",darrylbanks@yahoo.com,1949-06-15
4,ghardy,Mr. Gary Griffin,M,"65885 Amber Locks Apt. 633\nNorth Steven, ND 0...",brenda14@yahoo.com,1981-12-25
5,jcherry,Andrew Martinez,M,14704 Jennifer Extensions Suite 115\nNorth Ant...,andrewhowell@hotmail.com,1948-03-08
6,emyers,David Moody,M,"335 Gloria Street Suite 675\nMathisport, NJ 73764",webstermadeline@hotmail.com,1947-05-12
7,sarahdawson,Brandon Adams,M,"873 Amber Underpass\nMartinezfurt, AK 85986",joshua01@hotmail.com,1972-05-12


In [21]:
print(rows[0]) 

{'username': 'daniel02', 'name': 'Robert Duffy', 'sex': 'M', 'address': '6936 Griffin Extension Suite 018\nSouth Jim, SD 34941', 'mail': 'annahamilton@gmail.com', 'birthdate': '2020-10-01'}


Ahora vamos a localizar las claves del mail "joshua01@hotmail.com" a través del ínidice

In [24]:
rows = []
keys = bucket.get_index("idx_mail_bin", 'joshua01@hotmail.com')
for key in keys.results:
    row = bucket.get(key).data
    rows.append(row)
       
df = pd.read_json(json.dumps(rows))
df

,username,name,sex,address,mail,birthdate
0,sarahdawson,Brandon Adams,M,"873 Amber Underpass\nMartinezfurt, AK 85986",joshua01@hotmail.com,1972-05-12


## Listado de Buckets

In [25]:
for bucket in cliente.get_buckets():
        print('Bucket %s' % bucket )

Bucket <RiakBucket 'profile'>
Bucket <RiakBucket 'images'>
Bucket <RiakBucket 'bucket_name'>


## Listado de valores en un Bucket

In [26]:
for keys in bucket.stream_keys():
    print("Keys: ", keys)
    for key in keys:
        value = bucket.get(key).data
        print('Key %s: %s' % (key, value))

Keys:  ['kmartin']
Key kmartin: {'username': 'kmartin', 'name': 'Dwayne Miller', 'sex': 'M', 'address': '56403 Misty Knolls Suite 206\nMartinstad, ID 60623', 'mail': 'gregory32@hotmail.com', 'birthdate': '2016-10-11'}
Keys:  ['sarahdawson']
Key sarahdawson: {'username': 'sarahdawson', 'name': 'Brandon Adams', 'sex': 'M', 'address': '873 Amber Underpass\nMartinezfurt, AK 85986', 'mail': 'joshua01@hotmail.com', 'birthdate': '1972-05-12'}
Keys:  ['ghardy']
Key ghardy: {'username': 'ghardy', 'name': 'Mr. Gary Griffin', 'sex': 'M', 'address': '65885 Amber Locks Apt. 633\nNorth Steven, ND 09043', 'mail': 'brenda14@yahoo.com', 'birthdate': '1981-12-25'}
Keys:  ['daniel02']
Key daniel02: {'username': 'daniel02', 'name': 'Robert Duffy', 'sex': 'M', 'address': '6936 Griffin Extension Suite 018\nSouth Jim, SD 34941', 'mail': 'annahamilton@gmail.com', 'birthdate': '2020-10-01'}
Keys:  ['jcherry']
Key jcherry: {'username': 'jcherry', 'name': 'Andrew Martinez', 'sex': 'M', 'address': '14704 Jennifer

In [27]:
for key in bucket.get_keys():
    value = bucket.get(key).data
    print('Key %s: %s' % (key, value))

Key andersoncheryl: {'username': 'andersoncheryl', 'name': 'Ryan Roberts', 'sex': 'M', 'address': '1210 Juarez Crest Suite 970\nSolisburgh, DC 38745', 'mail': 'darrylbanks@yahoo.com', 'birthdate': '1949-06-15'}
Key key_three: {'val1': 1, 'val2': 'Dos', 'val3': [1, 2, 3, 4, 5]}
Key masonnguyen: {'username': 'masonnguyen', 'name': 'Scott Lucas', 'sex': 'M', 'address': '6401 Jeff Ford\nAngelicashire, WV 53209', 'mail': 'adam32@hotmail.com', 'birthdate': '1989-03-03'}
Key key_two: Dos
Key emyers: {'username': 'emyers', 'name': 'David Moody', 'sex': 'M', 'address': '335 Gloria Street Suite 675\nMathisport, NJ 73764', 'mail': 'webstermadeline@hotmail.com', 'birthdate': '1947-05-12'}
Key daniel02: {'username': 'daniel02', 'name': 'Robert Duffy', 'sex': 'M', 'address': '6936 Griffin Extension Suite 018\nSouth Jim, SD 34941', 'mail': 'annahamilton@gmail.com', 'birthdate': '2020-10-01'}
Key jamie41: {'username': 'jamie41', 'name': 'Brittany Miller', 'sex': 'F', 'address': '2871 Mary Ways Apt. 81